In [1]:
# read from preprocessed customer and customer test
from fastai.learner import *
from fastai.column_data import *
from fastai.structured import *

D:\python\envs\fastai\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
PATH=Path('data/elo')
# C drive path
PATH1 = Path('c:/data/')

In [3]:
customer = pd.read_feather(f'{PATH1}\customer')
customer_test = pd.read_csv(f'{PATH1}\customer_test1.csv')

In [4]:
customer.head()

,card_id,target,cust_index,purchase_amount,merch_index
0,C_ID_92a2005557,-0.820283,398,-0.707899,45766
1,C_ID_92a2005557,-0.820283,398,-0.641722,45766
2,C_ID_92a2005557,-0.820283,398,-0.734887,45766
3,C_ID_92a2005557,-0.820283,398,-0.713849,45766
4,C_ID_92a2005557,-0.820283,398,-0.641722,45766


In [14]:
customer_test.head()

,card_id,cust_index,merch_index,purchase_amount
0,C_ID_b3334831be,0,775,-2.846532
1,C_ID_af9b01c5b0,0,775,-2.846532
2,C_ID_22f6678098,0,775,-2.846532
3,C_ID_dcdf88cb4d,2,1896,-0.581616
4,C_ID_92b63a03a6,3,3499,-0.596643


In [6]:
del(customer_test['index'])

In [8]:
customer_test['cust_index'].max()

493.0

In [11]:
customer_test['cust_index'].fillna(0,inplace=True)

In [12]:
customer_test['merch_index'].fillna(0,inplace=True)

In [13]:
customer_test['cust_index'] = customer_test['cust_index'].astype(np.uint16)
customer_test['merch_index'] = customer_test['merch_index'].astype(np.uint16)

In [15]:
cat_vars = ['cust_index','merch_index']
contin_vars = ['purchase_amount']

In [32]:
from sklearn import preprocessing

def Normalizing(df):
    x = df['purchase_amount'].values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df['purchase_amount'] = pandas.DataFrame(x_scaled)
    return df 

In [33]:
customer = Normalizing(customer)

ValueError: Expected 2D array, got 1D array instead:
array=[-0.7079  -0.64172 -0.73489 ... -0.49146 -0.55156 -0.71386].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [16]:
dep = 'target'
df_train = customer[cat_vars+contin_vars+[dep]].copy()

In [17]:
customer_test[dep] = 0.0
df_test = customer_test[cat_vars+contin_vars+[dep, 'card_id']].copy()

In [18]:
for v in cat_vars: df_train[v] = df_train[v].astype('category').cat.as_ordered()
apply_cats(df_test, df_train)

In [19]:
for v in contin_vars:
    df_train[v] = df_train[v].fillna(0).astype('float32')
    df_test[v] = df_test[v].fillna(0).astype('float32')

In [21]:
samp_size = len(df_train)
df, y, nas, mapper = proc_df(df_train, dep, do_scale=True)

In [22]:
df_test1, _, nas, mapper = proc_df(df_test, dep, do_scale=True,skip_flds=['card_id'],mapper=mapper, na_dict=nas)

In [23]:
train_ratio = 0.75
# train_ratio = 0.9
train_size = int(samp_size * train_ratio); train_size
val_idx = list(range(train_size, len(df)))

In [24]:
def exp_rmse(y_pred, targ):
#     targ = inv_y(targ)
    pct_var = (targ - y_pred)
    return math.sqrt((pct_var**2).mean())

y_range = (0, 1)

In [25]:
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, y.astype(np.float32), cat_flds=cat_vars, bs=128,
                                       test_df=df_test1)

In [27]:
cat_sz = [(c, len(df_train[c].cat.categories)+1) for c in cat_vars]
cat_sz

[('cust_index', 496), ('merch_index', 38366)]

In [28]:
emb_szs = [(c, min(50, (c+1)//2)) for _,c in cat_sz]
emb_szs

[(496, 50), (38366, 50)]

In [29]:
m = md.get_learner(emb_szs,len(df.columns)-len(cat_vars),0.04,1,[1000,500],[0.001,0.01])

In [ ]:
lr=1e-3
m.lr_find()
m.sched.plot(100)

  2%|█                                                                   | 1381/86857 [00:14<14:38, 97.34it/s, loss=13]